<a href="https://colab.research.google.com/github/rajashekar/colab/blob/main/wake_word_detection_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Architecture & flow diagram - https://firebasestorage.googleapis.com/v0/b/firescript-577a2.appspot.com/o/imgs%2Fapp%2Frajashekar%2FQPitMq4MPE.png?alt=media&token=9daae34a-0c80-4920-91bc-bd03eb6661be

<img src="https://firebasestorage.googleapis.com/v0/b/firescript-577a2.appspot.com/o/imgs%2Fapp%2Frajashekar%2FQPitMq4MPE.png?alt=media&token=9daae34a-0c80-4920-91bc-bd03eb6661be" width="2500" height="500"/>

In [1]:
!nvidia-smi

Tue Jul 20 22:34:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!cat /proc/meminfo | head -1

MemTotal:       26696424 kB


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
%cd /content/gdrive/MyDrive/Colab/Fourth.Brain/wake_word_detection/howl

/content/gdrive/MyDrive/Colab/Fourth.Brain/wake_word_detection/howl


In [7]:
!python --version

Python 3.7.11


In [8]:
# install dependencies
# pocketsphinx requires https://github.com/bambocher/pocketsphinx-python#install-requirements
!sudo apt-get install -qq python python-dev python-pip build-essential swig git libpulse-dev libasound2-dev
# pyaudio
!sudo apt-get install python-pyaudio python3-pyaudio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-pyaudio is already the newest version (0.2.11-1build2).
python3-pyaudio is already the newest version (0.2.11-1build2).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [9]:
# unrar data
!unrar x -o- data.rar /content/

Streaming output truncated to the last 5000 lines.
Extracting  /content/data/hey-fourth-brain-stiched/audio/775.wav          96%  OK 
Extracting  /content/data/hey-fourth-brain-stiched/audio/7750.lab         96%  OK 
Extracting  /content/data/hey-fourth-brain-stiched/audio/7750.wav         96%  OK 
Extracting  /content/data/hey-fourth-brain-stiched/audio/7751.lab         96%  OK 
Extracting  /content/data/hey-fourth-brain-stiched/audio/7751.wav         96%  OK 
Extracting  /content/data/hey-fourth-brain-stiched/audio/7752.lab         96%  OK 
Extracting  /content/data/hey-fourth-brain-stiched/audio/7752.wav         96%  OK 
Extracting  /content/data/hey-fourth-brain-stiched/audio/7753.lab         96%  OK 
Extracting  /content/data/hey-fourth-brain-stiched/audio/7753.wav         96%  OK 
Extracting  /content/data/hey-fourth-brain-stiched/audio/7754.lab         96%  OK 
Extracting  /content/data/hey

In [10]:
# load data train meta data
import json
from pathlib import Path
def read_meta_data(path, jsonl_name):
  path = Path(path)
  metadata_list = []
  with open(path/jsonl_name) as f:
      for json_str in iter(f.readline, ""):
          metadata = json.loads(json_str)
          # metadata['path'] = f"{path}/audio/{metadata['path']}"
          metadata['path'] = (path / "audio" /  metadata['path']).absolute()
          metadata_list.append(metadata)
  return metadata_list

In [11]:
postive_data_path = "/content/data/hey-fourth-brain-positive/"
train_pos_metadata = read_meta_data(postive_data_path, "aligned-metadata-training.jsonl")
dev_pos_metadata = read_meta_data(postive_data_path, "aligned-metadata-dev.jsonl")
test_pos_metadata = read_meta_data(postive_data_path, "aligned-metadata-test.jsonl")

In [12]:
train_pos_metadata[0]

{'end_timestamps': [480.0,
  730.0,
  730.0,
  730.0,
  835.0,
  940.0,
  940.0,
  940.0,
  1021.25,
  1102.5,
  1183.75,
  1265.0,
  1346.25,
  1427.5,
  1508.75,
  1590.0,
  1590.0,
  1590.0,
  1663.3333333333333,
  1736.6666666666667,
  1810.0,
  1810.0,
  1810.0,
  1900.0,
  1990.0,
  1990.0,
  1990.0,
  2075.0,
  2160.0,
  2245.0,
  2330.0,
  2330.0,
  2330.0,
  2580.0,
  2580.0,
  2580.0,
  2625.0,
  2670.0,
  2670.0,
  2670.0,
  2776.0,
  2882.0,
  2988.0,
  3094.0,
  3200.0,
  3200.0,
  3200.0,
  3350.0,
  3500.0,
  3650.0,
  3650.0,
  3770.0,
  3838.0,
  3906.0,
  3974.0,
  4042.0,
  4110.0,
  4110.0,
  4110.0,
  4235.0,
  4360.0,
  4485.0,
  4610.0,
  4610.0,
  4610.0,
  4890.0,
  4890.0,
  4890.0,
  5015.0,
  5140.0,
  5265.0,
  5390.0,
  5515.0,
  5640.0],
 'path': PosixPath('/content/data/hey-fourth-brain-positive/audio/common_voice_en_20433916.wav'),
 'phone_end_timestamps': None,
 'phone_strings': None,
 'transcription': 'he was decorated with the order of the golden kit

In [13]:
print(f"Train samples in positive dataset {len(train_pos_metadata)}")
print(f"Dev samples in positive dataset {len(dev_pos_metadata)}")
print(f"Test samples in positive dataset {len(test_pos_metadata)}")

Train samples in positive dataset 828
Dev samples in positive dataset 29
Test samples in positive dataset 19


In [14]:
negative_data_path = "/content/data/hey-fourth-brain-negative/"
train_neg_metadata = read_meta_data(negative_data_path, "aligned-metadata-training.jsonl")
dev_neg_metadata = read_meta_data(negative_data_path, "aligned-metadata-dev.jsonl")
test_neg_metadata = read_meta_data(negative_data_path, "aligned-metadata-test.jsonl")

In [15]:
print(f"Train samples in negative dataset {len(train_neg_metadata)}")
print(f"Dev samples in negative dataset {len(dev_neg_metadata)}")
print(f"Test samples in negative dataset {len(test_neg_metadata)}")

Train samples in negative dataset 11254
Dev samples in negative dataset 322
Test samples in negative dataset 312


In [16]:
stiched_data_path = "/content/data/hey-fourth-brain-stiched/"
train_stiched_metadata = read_meta_data(stiched_data_path, "aligned-metadata-training.jsonl")
dev_stiched_metadata = read_meta_data(stiched_data_path, "aligned-metadata-dev.jsonl")
test_stiched_metadata = read_meta_data(stiched_data_path, "aligned-metadata-test.jsonl")

In [17]:
print(f"Train samples in stiched dataset {len(train_stiched_metadata)}")
print(f"Dev samples in stiched dataset {len(dev_stiched_metadata)}")
print(f"Test samples in stiched dataset {len(test_stiched_metadata)}")

Train samples in stiched dataset 5000
Dev samples in stiched dataset 2500
Test samples in stiched dataset 2500


In [18]:
train_ds = train_pos_metadata + train_neg_metadata + train_stiched_metadata
dev_ds = dev_pos_metadata + dev_neg_metadata + dev_stiched_metadata
test_ds = test_pos_metadata + test_neg_metadata + test_stiched_metadata

In [19]:
print(f"Total Train samples {len(train_ds)}")
print(f"Total Dev samples {len(dev_ds)}")
print(f"Total test samples {len(test_ds)}")

Total Train samples 17082
Total Dev samples 2851
Total test samples 2831


In [20]:
vocab = ["hey", "fourth", "brain"]
inference_sequence = [0, 1, 2]
vocab_map = dict(zip(vocab, inference_sequence))

In [21]:
inference_sequence_str = ''.join(map(str, inference_sequence))
def search_vocab(transcription):
  exists = False
  final = ""
  for word in transcription.lower().split():
    if word in vocab:
      final += str(vocab_map[word])
    else:
      final += str(len(inference_sequence))
  
  if final == inference_sequence_str:
    exists = True
  return exists

In [22]:
# dev containing vocab
dev_pos_ds = list(filter(lambda x: search_vocab(x['transcription']), dev_ds))
# dev NOT containing vocab
dev_neg_ds = list(filter(lambda x: not search_vocab(x['transcription']), dev_ds))
# test containing vocab
test_pos_ds = list(filter(lambda x: search_vocab(x['transcription']), test_ds))
# test NOT containing vocab
test_neg_ds = list(filter(lambda x: not search_vocab(x['transcription']), test_ds))

In [23]:
test_pos_ds[0]

{'end_timestamps': [0.0,
  35.0,
  70.0,
  70.0,
  70.0,
  140.0,
  210.0,
  280.0,
  349.9999999999998,
  419.9999999999998,
  419.9999999999998,
  419.9999999999998,
  524.9999999999998,
  629.9999999999998,
  734.9999999999998,
  839.9999999999998],
 'path': PosixPath('/content/data/hey-fourth-brain-stiched/audio/6010.wav'),
 'phone_end_timestamps': None,
 'phone_strings': None,
 'transcription': 'hey fourth brain',
 'word_end_timestamps': None,
 'words': None}

In [50]:
import librosa
import warnings

def load_audio_data(path: str, sr: int = 16000, mono: bool = True):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        return librosa.core.load(path, sr=sr, mono=mono)[0]

In [51]:
def compute_labels(metadata):
        frame_labels = dict()
        start_timestamp = []
        char_indices = []

        char_idx = 0
        for word in metadata["transcription"].lower().split():
          if word in vocab:
            vocab_found = True
            word_size = len(word.rstrip())

            # if the current word is in vocab, store necessary informations
            if vocab_found:
                label = vocab_map[word]
                end_timestamp = metadata['end_timestamps'][char_idx + word_size - 1]
                frame_labels[end_timestamp] = label
                char_indices.append((label, list(range(char_idx, char_idx + word_size))))
                start_timestamp.append((label, metadata['end_timestamps'][char_idx - 1] if char_idx > 0 else 0.0))

            char_idx += word_size + 1  # space

        return  {
            "timestamp_label_map": frame_labels,
            "start_timestamp": start_timestamp,
            "char_indices": char_indices
        }

In [52]:
import random
import torch
import numpy as np

def process_batch(batch):
    sample_rate = 16000
    window_size_ms = 750
    negative_label = len(vocab)
    examples = []
    max_window_size = 16000
    max_length = int(window_size_ms / 1000 * sample_rate)

    for sample in batch:
      example = {}
      example['audio_data'] = torch.from_numpy(load_audio_data(sample['path']))
      example['label_data'] = compute_labels(sample)
      # if the sample does not contain any sample value
      if not example['label_data']["timestamp_label_map"]:
        if example['audio_data'].size(-1) < max_length:
          examples.append((negative_label, example['audio_data']))
        else:
          a = random.randint(0, example['audio_data'].size(-1) - max_length)
          examples.append((negative_label, example['audio_data'][..., a:a + max_window_size]))
        continue
      
      end_ms, label = random.choice(list( example['label_data']["timestamp_label_map"].items()))
      end_ms_rand = end_ms + (random.random() * 20)
      b = int((end_ms_rand / 1000) * sample_rate)
      a = max(b - int((window_size_ms / 1000) * sample_rate), 0)
      examples.append((label, example['audio_data'][..., a:b])) 
    
    labels_lst = [x[0] for x in examples]
    audio_ds = [x[1] for x in examples]
    lengths=[x[1].size(-1) for x in examples]

    lengths_nd = np.array(lengths)
    sort_indices = np.argsort(-lengths_nd)
    audio_data_lst = np.array(audio_ds, dtype=object)[sort_indices].tolist()
    labels_lst = np.array(labels_lst, dtype=object)[sort_indices].tolist()
    lengths = np.array(lengths, dtype=object)[sort_indices].tolist()
    
    audio_tensor = []
    for audio_data in audio_data_lst:
      if audio_data.size(-1) > max_length:
        audio_data = audio_data[:max_length]
      squeezed_data = audio_data.squeeze()
      if squeezed_data.dim() == 0:
          squeezed_data = squeezed_data.unsqueeze(0)
      if random.random() < 0.5:
          x = (torch.zeros(max_length - audio_data.size(-1)), squeezed_data)
      else:
          x = (squeezed_data, torch.zeros(max_length - audio_data.size(-1)))
      audio_tensor.append(torch.cat(x, -1)) 


    batch_tensor = {
      'audio_data' : torch.stack(audio_tensor),
      'labels' : torch.tensor(labels_lst),
      'lengths' : torch.tensor(lengths)
    }

    return batch_tensor
    

In [53]:
import torch.utils.data as tud

batch_size = 16
num_workers = 0

train_dl = tud.DataLoader(train_ds,
                  batch_size=batch_size,
                  drop_last=True,
                  shuffle=True,
                  num_workers=num_workers,
                  collate_fn=process_batch)

In [54]:
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('CUDA is not available.  Inference on CPU ...')
else:
    print('CUDA is available!  Inference on GPU ...')

CUDA is available!  Inference on GPU ...


In [55]:
import torch.nn as nn
import torch.nn.functional as F


class CNN(nn.Module):
    def __init__(self, num_labels, num_maps1, num_maps2, num_hidden_input, hidden_size):
        super(CNN, self).__init__()
        conv0 = nn.Conv2d(1, num_maps1, (8, 16), padding=(4, 0), stride=(2, 2), bias=True)
        pool = nn.MaxPool2d(2)
        conv1 = nn.Conv2d(num_maps1, num_maps2, (5, 5), padding=2, stride=(2, 1), bias=True)
        self.encoder1 = nn.Sequential(conv0,
                                      nn.ReLU(),
                                      pool,
                                      nn.BatchNorm2d(num_maps1, affine=True))
        self.encoder2 = nn.Sequential(conv1,
                                      nn.ReLU(),
                                      pool,
                                      nn.BatchNorm2d(num_maps2, affine=True))
        self.output = nn.Sequential(nn.Linear(num_hidden_input, hidden_size),
                                    nn.ReLU(),
                                    nn.Dropout(0.1),
                                    nn.Linear(hidden_size, num_labels))

    def forward(self, x, lengths):
        x = x[:, :1]  # log-Mels only
        x = x.permute(0, 1, 3, 2)  # (time, frequency)
        x1 = self.encoder1(x)
        x2 = self.encoder2(x1)
        x2 = x2.view(x2.size(0), x2.size(1), -1)
        x = x2.view(x2.size(0), -1)
        return self.output(x)

In [56]:
num_labels = len(vocab) + 1 # oov
num_maps1  = 48
num_maps2  = 64
num_hidden_input =  768
hidden_size = 128
model = CNN(num_labels, num_maps1, num_maps2, num_hidden_input, hidden_size)

In [57]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
model

CNN(
  (encoder1): Sequential(
    (0): Conv2d(1, 48, kernel_size=(8, 16), stride=(2, 2), padding=(4, 0))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (encoder2): Sequential(
    (0): Conv2d(48, 64, kernel_size=(5, 5), stride=(2, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (output): Sequential(
    (0): Linear(in_features=768, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=128, out_features=4, bias=True)
  )
)

In [58]:
from torch.optim.adamw import AdamW

learning_rate = 0.001
weight_decay = 0.0
lr_decay = 0.75

criterion = nn.CrossEntropyLoss()
params = list(filter(lambda x: x.requires_grad, model.parameters()))
optimizer = AdamW(params, learning_rate, weight_decay=weight_decay)

In [59]:
pip install torchaudio

In [60]:
from torchaudio.transforms import MelSpectrogram, ComputeDeltas

epochs = 2
num_mels = 40 # https://en.wikipedia.org/wiki/Mel_scale
num_fft = 512 # window length - Fast Fourier Transform
hop_length = 200 
sample_rate = 16000

for epoch in range(epochs):
  model.train()
  total_loss = torch.Tensor([0.0]).to(device)

  for batch in train_dl:
    audio_data = batch['audio_data'].to(device)
    labels = batch['labels'].to(device)
    lengths = batch['lengths'].to(device)

    # Transformations
    # Mel-scale spectrogram is a combination of Spectrogram and mel scale conversion
    mel_spectrogram  = MelSpectrogram(n_mels=num_mels,
                                      sample_rate=sample_rate,
                                      n_fft=num_fft,
                                      hop_length=hop_length)
    mel_spectrogram.to(device)
    log_mels = mel_spectrogram(audio_data).add_(1e-7).log_().contiguous()

    # Compute delta coefficients of a tensor, usually a spectrogram.
    compute_deltas = ComputeDeltas()
    deltas = compute_deltas(log_mels)
    accels = compute_deltas(deltas)

    mel_audio_data = torch.stack((log_mels, deltas, accels), 1).to(device)
    lengths = ((lengths - num_fft) // hop_length + 1).long()

    # Train
    predicted_scores = model(mel_audio_data, lengths)
    # get loss
    loss = criterion(predicted_scores, labels)

    optimizer.zero_grad()
    model.zero_grad()

    # backward propagation
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        total_loss += loss
      
  for group in optimizer.param_groups:
    group["lr"] *= lr_decay

  mean = total_loss / len(train_dl)
  print(f"Epoch {epoch}: Training loss {mean.item()} with lr {group['lr']}")
  
  # Evaluate  

Epoch 0: Training loss 0.2580280005931854 with lr 0.00075
Epoch 1: Training loss 0.1621510088443756 with lr 0.0005625000000000001
